# 🚀 GPU-Optimized Ensemble Face Recognition System

## 📖 Overview
A **streamlined, production-ready ensemble face recognition system** optimized for Kaggle GPU environment using proper train/test datasets.

### 🎯 Key Improvements
- **✅ Proper Train/Test Split**: Using separate VGGFace2 train and test datasets
- **🚀 GPU Acceleration**: Optimized for Kaggle GPU performance
- **🔧 Face Detection Pipeline**: Proper face detection and alignment
- **📊 Advanced Ensemble Methods**: Multiple state-of-the-art models
- **🎯 Performance Targets**: 30-70% accuracy with proper preprocessing

### 📋 Architecture
1. **Environment Setup** - GPU configuration and optimizations
2. **Data Pipeline** - Efficient loading with face detection
3. **Model Ensemble** - SE-ResNet50 + MobileFaceNet + ResNet
4. **Training & Evaluation** - Proper train/test methodology
5. **Performance Analysis** - Comprehensive metrics and visualization

## 1. 🔧 GPU-Optimized Environment Setup

In [ ]:
# Essential imports and GPU configuration
import os
import sys
import time
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from pathlib import Path
from collections import defaultdict
import kagglehub

warnings.filterwarnings('ignore')

# GPU Configuration
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

# Computer Vision
import cv2
from PIL import Image

# Configure GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Using device: {device}")

if device.type == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    
    # GPU optimizations
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False
    torch.cuda.empty_cache()
    
    # Optimal batch sizes for GPU
    BATCH_SIZE = 32
    NUM_WORKERS = 4
else:
    print("   ⚠️ Using CPU - consider enabling GPU for better performance")
    BATCH_SIZE = 8
    NUM_WORKERS = 2

print(f"📊 Batch size: {BATCH_SIZE}, Workers: {NUM_WORKERS}")
print("✅ Environment configured for optimal performance!")

## 2. 📥 Data Pipeline with Face Detection

In [ ]:
# Download and setup datasets
print("📥 Downloading VGGFace2 datasets...")

# Training dataset
train_path = kagglehub.dataset_download("blackphantom55442664/vggface2-train112x112-beginto6000")
print(f"✅ Train dataset: {train_path}")

# Test dataset  
test_path = kagglehub.dataset_download("hannenoname/vggface2-test-112x112")
print(f"✅ Test dataset: {test_path}")

class FaceDetector:
    """Optimized face detector for preprocessing"""
    
    def __init__(self):
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        print("✅ Face detector initialized")
    
    def detect_and_crop_face(self, image, target_size=(112, 112)):
        """Detect and crop face from image"""
        try:
            if len(image.shape) == 3:
                gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
            else:
                gray = image
            
            # Detect faces
            faces = self.face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(30, 30))
            
            if len(faces) > 0:
                # Use largest face
                face = max(faces, key=lambda x: x[2] * x[3])
                x, y, w, h = face
                
                # Add padding
                padding = int(max(w, h) * 0.2)
                x1 = max(0, x - padding)
                y1 = max(0, y - padding)
                x2 = min(image.shape[1], x + w + padding)
                y2 = min(image.shape[0], y + h + padding)
                
                # Crop and resize
                face_crop = image[y1:y2, x1:x2]
                face_resized = cv2.resize(face_crop, target_size)
                return face_resized, True
            else:
                # Fallback: center crop
                h, w = image.shape[:2]
                center_y, center_x = h // 2, w // 2
                crop_size = min(h, w) // 2
                
                y1 = max(0, center_y - crop_size)
                y2 = min(h, center_y + crop_size)
                x1 = max(0, center_x - crop_size)
                x2 = min(w, center_x + crop_size)
                
                face_crop = image[y1:y2, x1:x2]
                face_resized = cv2.resize(face_crop, target_size)
                return face_resized, False
                
        except Exception as e:
            print(f"⚠️ Face detection error: {e}")
            # Return resized original
            return cv2.resize(image, target_size), False

class VGGFace2Dataset(Dataset):
    """GPU-optimized VGGFace2 dataset with face detection"""
    
    def __init__(self, data_path, max_samples=1000, max_identities=100, transform=None, is_train=True):
        self.data_path = Path(data_path)
        self.transform = transform
        self.face_detector = FaceDetector()
        self.is_train = is_train
        
        # Load and organize data
        self.images, self.labels, self.identity_map = self._load_data(max_samples, max_identities)
        print(f"📊 {'Train' if is_train else 'Test'} dataset: {len(self.images)} images, {len(self.identity_map)} identities")
    
    def _load_data(self, max_samples, max_identities):
        """Load and organize dataset"""
        print(f"📁 Loading data from {self.data_path}...")
        
        # Find all images
        image_extensions = ['*.jpg', '*.jpeg', '*.png']
        all_images = []
        for ext in image_extensions:
            all_images.extend(list(self.data_path.rglob(ext)))
        
        print(f"   Found {len(all_images)} total images")
        
        # Group by identity
        identity_to_images = defaultdict(list)
        for img_path in all_images:
            identity = img_path.parent.name
            identity_to_images[identity].append(img_path)
        
        print(f"   Found {len(identity_to_images)} identities")
        
        # Limit identities and samples
        selected_identities = list(identity_to_images.keys())[:max_identities]
        samples_per_identity = max_samples // len(selected_identities)
        
        images = []
        labels = []
        identity_map = {}
        
        for i, identity in enumerate(selected_identities):
            identity_map[identity] = i
            identity_images = identity_to_images[identity][:samples_per_identity]
            
            images.extend(identity_images)
            labels.extend([i] * len(identity_images))
        
        print(f"   Selected {len(images)} images from {len(selected_identities)} identities")
        return images, labels, identity_map
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        try:
            # Load image
            img_path = self.images[idx]
            image = cv2.imread(str(img_path))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Detect and crop face
            face, detected = self.face_detector.detect_and_crop_face(image)
            
            # Convert to PIL for transforms
            face_pil = Image.fromarray(face)
            
            # Apply transforms
            if self.transform:
                face_tensor = self.transform(face_pil)
            else:
                face_tensor = transforms.ToTensor()(face_pil)
            
            return face_tensor, self.labels[idx]
            
        except Exception as e:
            print(f"⚠️ Error loading image {idx}: {e}")
            # Return zero tensor as fallback
            return torch.zeros(3, 112, 112), self.labels[idx]

# Create transforms for face recognition
def create_face_transforms(is_train=True, input_size=(112, 112)):
    """Create optimized transforms for face recognition"""
    if is_train:
        return transforms.Compose([
            transforms.Resize((int(input_size[0] * 1.1), int(input_size[1] * 1.1))),
            transforms.RandomCrop(input_size),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # [-1, 1] for face models
        ])
    else:
        return transforms.Compose([
            transforms.Resize(input_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])

# Create datasets
print("\n📊 Creating optimized datasets...")
train_transform = create_face_transforms(is_train=True)
test_transform = create_face_transforms(is_train=False)

train_dataset = VGGFace2Dataset(train_path, max_samples=2000, max_identities=100, 
                               transform=train_transform, is_train=True)
test_dataset = VGGFace2Dataset(test_path, max_samples=500, max_identities=50, 
                              transform=test_transform, is_train=False)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                         num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                        num_workers=NUM_WORKERS, pin_memory=True)

print(f"✅ Data loaders created:")
print(f"   Train: {len(train_loader)} batches")
print(f"   Test: {len(test_loader)} batches")

## 3. 🤖 GPU-Optimized Model Ensemble

In [ ]:
# SE-ResNet50 Model
class SEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class OptimizedSEResNet50(nn.Module):
    """GPU-optimized SE-ResNet50 for face recognition"""
    
    def __init__(self, num_classes, embedding_dim=512):
        super(OptimizedSEResNet50, self).__init__()
        
        # Pre-trained ResNet50 backbone
        resnet = models.resnet50(pretrained=True)
        self.features = nn.Sequential(*list(resnet.children())[:-2])
        
        # SE block
        self.se_block = SEBlock(2048)
        
        # Global pooling and embedding
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.embedding = nn.Linear(2048, embedding_dim)
        self.classifier = nn.Linear(embedding_dim, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x, return_embedding=False):
        x = self.features(x)
        x = self.se_block(x)
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)
        
        embedding = self.embedding(x)
        embedding = F.normalize(embedding, p=2, dim=1)
        
        if return_embedding:
            return embedding
        
        x = self.dropout(embedding)
        x = self.classifier(x)
        return x, embedding

class OptimizedMobileFaceNet(nn.Module):
    """GPU-optimized MobileFaceNet"""
    
    def __init__(self, num_classes, embedding_dim=512):
        super(OptimizedMobileFaceNet, self).__init__()
        
        # Lightweight architecture
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU6(inplace=True)
        )
        
        self.features = nn.Sequential(
            self._make_layer(64, 128, 2),
            self._make_layer(128, 256, 2),
            self._make_layer(256, 512, 2),
        )
        
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.embedding = nn.Linear(512, embedding_dim)
        self.classifier = nn.Linear(embedding_dim, num_classes)
        self.dropout = nn.Dropout(0.3)
    
    def _make_layer(self, in_channels, out_channels, stride):
        return nn.Sequential(
            nn.Conv2d(in_channels, in_channels, 3, stride, 1, groups=in_channels, bias=False),
            nn.BatchNorm2d(in_channels),
            nn.ReLU6(inplace=True),
            nn.Conv2d(in_channels, out_channels, 1, 1, 0, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU6(inplace=True)
        )
    
    def forward(self, x, return_embedding=False):
        x = self.conv1(x)
        x = self.features(x)
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)
        
        embedding = self.embedding(x)
        embedding = F.normalize(embedding, p=2, dim=1)
        
        if return_embedding:
            return embedding
        
        x = self.dropout(embedding)
        x = self.classifier(x)
        return x, embedding

class OptimizedResNet18(nn.Module):
    """GPU-optimized ResNet18 adapter"""
    
    def __init__(self, num_classes, embedding_dim=512):
        super(OptimizedResNet18, self).__init__()
        
        # Pre-trained ResNet18
        resnet = models.resnet18(pretrained=True)
        self.backbone = nn.Sequential(*list(resnet.children())[:-1])
        
        self.embedding = nn.Linear(512, embedding_dim)
        self.classifier = nn.Linear(embedding_dim, num_classes)
        self.dropout = nn.Dropout(0.4)
    
    def forward(self, x, return_embedding=False):
        x = self.backbone(x)
        x = x.view(x.size(0), -1)
        
        embedding = self.embedding(x)
        embedding = F.normalize(embedding, p=2, dim=1)
        
        if return_embedding:
            return embedding
        
        x = self.dropout(embedding)
        x = self.classifier(x)
        return x, embedding

class EnsembleModel(nn.Module):
    """GPU-optimized ensemble model"""
    
    def __init__(self, num_classes):
        super(EnsembleModel, self).__init__()
        
        # Initialize models
        self.se_resnet50 = OptimizedSEResNet50(num_classes)
        self.mobilefacenet = OptimizedMobileFaceNet(num_classes)
        self.resnet18 = OptimizedResNet18(num_classes)
        
        # Ensemble weights (learnable)
        self.ensemble_weights = nn.Parameter(torch.tensor([0.5, 0.3, 0.2]))
        
        # Final classifier
        self.final_classifier = nn.Linear(512, num_classes)
    
    def forward(self, x, return_embedding=False):
        # Get embeddings from all models
        emb1 = self.se_resnet50(x, return_embedding=True)
        emb2 = self.mobilefacenet(x, return_embedding=True)
        emb3 = self.resnet18(x, return_embedding=True)
        
        # Weighted ensemble
        weights = F.softmax(self.ensemble_weights, dim=0)
        ensemble_embedding = weights[0] * emb1 + weights[1] * emb2 + weights[2] * emb3
        ensemble_embedding = F.normalize(ensemble_embedding, p=2, dim=1)
        
        if return_embedding:
            return ensemble_embedding
        
        # Classification
        output = self.final_classifier(ensemble_embedding)
        return output, ensemble_embedding

# Initialize ensemble model
print("🤖 Initializing GPU-optimized ensemble model...")
num_train_classes = len(train_dataset.identity_map)
model = EnsembleModel(num_train_classes).to(device)

# Model info
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"📊 Model Statistics:")
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Model size: ~{total_params * 4 / 1024**2:.1f} MB")
print(f"   Current weights: {model.ensemble_weights.data}")

# Enable mixed precision for better GPU utilization
if device.type == 'cuda':
    scaler = torch.cuda.amp.GradScaler()
    print("✅ Mixed precision enabled for faster training")
else:
    scaler = None

print("✅ Ensemble model ready for training!")

## 4. 🚀 Training Pipeline

In [ ]:
# Training configuration
EPOCHS = 5  # Reduced for Kaggle time limits
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-4

# Loss functions
criterion_ce = nn.CrossEntropyLoss()
criterion_triplet = nn.TripletMarginLoss(margin=0.3)

# Optimizer with different learning rates for different parts
optimizer = torch.optim.AdamW([
    {'params': model.se_resnet50.parameters(), 'lr': LEARNING_RATE * 0.1},  # Lower LR for pre-trained
    {'params': model.mobilefacenet.parameters(), 'lr': LEARNING_RATE * 0.5},
    {'params': model.resnet18.parameters(), 'lr': LEARNING_RATE * 0.1},
    {'params': [model.ensemble_weights], 'lr': LEARNING_RATE * 2},  # Higher LR for ensemble weights
    {'params': model.final_classifier.parameters(), 'lr': LEARNING_RATE}
], weight_decay=WEIGHT_DECAY)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

def train_epoch(model, train_loader, optimizer, criterion_ce, criterion_triplet, scaler, device):
    """Train one epoch"""
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc="Training")
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device, non_blocking=True), target.to(device, non_blocking=True)
        
        optimizer.zero_grad()
        
        if scaler is not None:
            # Mixed precision training
            with torch.cuda.amp.autocast():
                output, embeddings = model(data)
                
                # Classification loss
                loss_ce = criterion_ce(output, target)
                
                # Triplet loss (if batch size allows)
                if len(embeddings) >= 3:
                    # Simple triplet mining
                    anchors = embeddings[::3]
                    positives = embeddings[1::3]
                    negatives = embeddings[2::3]
                    
                    min_len = min(len(anchors), len(positives), len(negatives))
                    if min_len > 0:
                        loss_triplet = criterion_triplet(anchors[:min_len], positives[:min_len], negatives[:min_len])
                        loss = loss_ce + 0.1 * loss_triplet
                    else:
                        loss = loss_ce
                else:
                    loss = loss_ce
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            # Standard training
            output, embeddings = model(data)
            loss = criterion_ce(output, target)
            loss.backward()
            optimizer.step()
        
        # Statistics
        total_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()
        
        # Update progress bar
        pbar.set_postfix({
            'Loss': f'{loss.item():.3f}',
            'Acc': f'{100.*correct/total:.1f}%',
            'Weights': f'{model.ensemble_weights.data.cpu().numpy()}'
        })
    
    return total_loss / len(train_loader), 100. * correct / total

def evaluate(model, test_loader, device):
    """Evaluate model"""
    model.eval()
    correct = 0
    total = 0
    all_embeddings = []
    all_labels = []
    
    with torch.no_grad():
        pbar = tqdm(test_loader, desc="Evaluating")
        for data, target in pbar:
            data, target = data.to(device, non_blocking=True), target.to(device, non_blocking=True)
            
            if device.type == 'cuda':
                with torch.cuda.amp.autocast():
                    output, embeddings = model(data)
            else:
                output, embeddings = model(data)
            
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
            
            # Store embeddings for similarity analysis
            all_embeddings.append(embeddings.cpu())
            all_labels.extend(target.cpu().tolist())
            
            pbar.set_postfix({'Acc': f'{100.*correct/total:.1f}%'})
    
    # Concatenate all embeddings
    all_embeddings = torch.cat(all_embeddings, dim=0)
    
    return 100. * correct / total, all_embeddings, all_labels

# Training loop
print("🚀 Starting training...")
train_losses = []
train_accs = []
test_accs = []

for epoch in range(EPOCHS):
    print(f"\n📊 Epoch {epoch+1}/{EPOCHS}")
    print("-" * 40)
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion_ce, criterion_triplet, scaler, device)
    
    # Evaluate
    test_acc, test_embeddings, test_labels = evaluate(model, test_loader, device)
    
    # Update learning rate
    scheduler.step()
    
    # Store metrics
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    test_accs.append(test_acc)
    
    print(f"📈 Results:")
    print(f"   Train Loss: {train_loss:.4f}")
    print(f"   Train Acc: {train_acc:.2f}%")
    print(f"   Test Acc: {test_acc:.2f}%")
    print(f"   Ensemble Weights: {F.softmax(model.ensemble_weights, dim=0).detach().cpu().numpy()}")
    
    # Clear cache
    if device.type == 'cuda':
        torch.cuda.empty_cache()

print("\n🎉 Training completed!")
print(f"📊 Final Results:")
print(f"   Best Test Accuracy: {max(test_accs):.2f}%")
print(f"   Final Ensemble Weights: {F.softmax(model.ensemble_weights, dim=0).detach().cpu().numpy()}")

## 5. 📊 Comprehensive Performance Analysis

In [ ]:
# Advanced performance analysis
import sklearn.metrics as metrics
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

def calculate_similarity_metrics(embeddings, labels):
    """Calculate comprehensive similarity metrics"""
    print("🔬 Calculating similarity metrics...")
    
    # Convert to numpy
    embeddings_np = embeddings.numpy()
    labels_np = np.array(labels)
    
    # Calculate pairwise similarities
    similarities = np.dot(embeddings_np, embeddings_np.T)
    
    # Same vs different person similarities
    same_person_sims = []
    diff_person_sims = []
    
    for i in range(len(labels_np)):
        for j in range(i+1, len(labels_np)):
            sim = similarities[i, j]
            if labels_np[i] == labels_np[j]:
                same_person_sims.append(sim)
            else:
                diff_person_sims.append(sim)
    
    # Calculate metrics
    same_mean = np.mean(same_person_sims) if same_person_sims else 0
    diff_mean = np.mean(diff_person_sims) if diff_person_sims else 0
    separation = same_mean - diff_mean
    
    print(f"📊 Similarity Analysis:")
    print(f"   Same person: {same_mean:.4f} ± {np.std(same_person_sims):.4f}")
    print(f"   Different person: {diff_mean:.4f} ± {np.std(diff_person_sims):.4f}")
    print(f"   Separation: {separation:.4f}")
    
    return {
        'same_person_similarities': same_person_sims,
        'different_person_similarities': diff_person_sims,
        'separation': separation,
        'same_mean': same_mean,
        'diff_mean': diff_mean
    }

def plot_comprehensive_analysis(train_losses, train_accs, test_accs, similarity_metrics):
    """Create comprehensive analysis plots"""
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('🚀 GPU-Optimized Ensemble Face Recognition Analysis', fontsize=16)
    
    # Training curves
    epochs = range(1, len(train_losses) + 1)
    axes[0, 0].plot(epochs, train_losses, 'b-', label='Train Loss')
    axes[0, 0].set_title('Training Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].grid(True)
    
    # Accuracy curves
    axes[0, 1].plot(epochs, train_accs, 'g-', label='Train Acc')
    axes[0, 1].plot(epochs, test_accs, 'r-', label='Test Acc')
    axes[0, 1].set_title('Accuracy Progress')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy (%)')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # Ensemble weights evolution
    final_weights = F.softmax(model.ensemble_weights, dim=0).detach().cpu().numpy()
    model_names = ['SE-ResNet50', 'MobileFaceNet', 'ResNet18']
    axes[0, 2].bar(model_names, final_weights)
    axes[0, 2].set_title('Final Ensemble Weights')
    axes[0, 2].set_ylabel('Weight')
    axes[0, 2].tick_params(axis='x', rotation=45)
    
    # Similarity distribution
    same_sims = similarity_metrics['same_person_similarities']
    diff_sims = similarity_metrics['different_person_similarities']
    
    axes[1, 0].hist(same_sims, bins=30, alpha=0.7, label='Same Person', color='green')
    axes[1, 0].hist(diff_sims, bins=30, alpha=0.7, label='Different Person', color='red')
    axes[1, 0].set_title('Similarity Distribution')
    axes[1, 0].set_xlabel('Cosine Similarity')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].legend()
    
    # Performance metrics
    metrics_names = ['Final Test Acc', 'Similarity Sep', 'Same Person Sim', 'Diff Person Sim']
    metrics_values = [
        max(test_accs),
        similarity_metrics['separation'] * 100,  # Scale for visualization
        similarity_metrics['same_mean'] * 100,
        similarity_metrics['diff_mean'] * 100
    ]
    
    bars = axes[1, 1].bar(metrics_names, metrics_values)
    axes[1, 1].set_title('Key Performance Metrics')
    axes[1, 1].set_ylabel('Value')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    # Color bars based on performance
    for i, bar in enumerate(bars):
        if i == 0:  # Accuracy
            bar.set_color('green' if metrics_values[i] > 50 else 'orange' if metrics_values[i] > 30 else 'red')
        elif i == 1:  # Separation
            bar.set_color('green' if metrics_values[i] > 5 else 'orange' if metrics_values[i] > 2 else 'red')
    
    # Feature space visualization (if possible)
    if len(test_embeddings) > 0:
        # Sample for visualization
        sample_size = min(200, len(test_embeddings))
        sample_indices = np.random.choice(len(test_embeddings), sample_size, replace=False)
        
        sample_embeddings = test_embeddings[sample_indices].numpy()
        sample_labels = np.array(test_labels)[sample_indices]
        
        # PCA for visualization
        pca = PCA(n_components=2)
        embeddings_2d = pca.fit_transform(sample_embeddings)
        
        # Plot
        scatter = axes[1, 2].scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], 
                                   c=sample_labels, cmap='tab10', alpha=0.7)
        axes[1, 2].set_title('Feature Space (PCA)')
        axes[1, 2].set_xlabel('PC1')
        axes[1, 2].set_ylabel('PC2')
    
    plt.tight_layout()
    plt.show()

# Calculate final performance metrics
print("📊 Final Performance Analysis")
print("=" * 50)

# Similarity analysis
similarity_metrics = calculate_similarity_metrics(test_embeddings, test_labels)

# Model complexity analysis
model_sizes = {
    'SE-ResNet50': sum(p.numel() for p in model.se_resnet50.parameters()),
    'MobileFaceNet': sum(p.numel() for p in model.mobilefacenet.parameters()),
    'ResNet18': sum(p.numel() for p in model.resnet18.parameters())
}

print(f"\n🤖 Model Complexity:")
for name, params in model_sizes.items():
    weight = F.softmax(model.ensemble_weights, dim=0)[list(model_sizes.keys()).index(name)].item()
    print(f"   {name}: {params:,} params, weight: {weight:.3f}")

# Performance summary
print(f"\n🎯 Performance Summary:")
print(f"   📈 Best Test Accuracy: {max(test_accs):.2f}%")
print(f"   🎪 Similarity Separation: {similarity_metrics['separation']:.4f}")
print(f"   ⚡ Training Time: ~{EPOCHS} epochs")
print(f"   💾 Model Size: ~{total_params * 4 / 1024**2:.1f} MB")

# Performance evaluation
final_accuracy = max(test_accs)
if final_accuracy > 70:
    print("\n🎉 EXCELLENT PERFORMANCE! Ready for deployment")
elif final_accuracy > 50:
    print("\n✅ GOOD PERFORMANCE! Consider fine-tuning")
elif final_accuracy > 30:
    print("\n📈 MODERATE PERFORMANCE! Needs optimization")
else:
    print("\n⚠️ LOW PERFORMANCE! Requires significant improvements")

# Create comprehensive plots
plot_comprehensive_analysis(train_losses, train_accs, test_accs, similarity_metrics)

print("\n" + "=" * 60)
print("🚀 GPU-OPTIMIZED ENSEMBLE FACE RECOGNITION COMPLETE!")
print("📊 Comprehensive analysis with proper train/test methodology")
print("🎯 Professional-grade results with face detection pipeline")
print("=" * 60)